This notebook is for exploring and clustering the neighborhoods in Toronto.

Importing the libraries:

In [5]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

Parsing Wikipedia page and extracting table from it:

In [6]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(website_url, 'lxml')
#print(soup.prettify())
mytable = soup.find('table',{'class':'wikitable sortable'})

Find all _< tr >_ tags in the table:

In [7]:
trs=mytable.find_all('tr')

Create a list of the values inside the _< tr >_ tags:

In [8]:
values=[]
for tr in trs:
    values.append(tr.text.rstrip())
values

['\nPostcode\nBorough\nNeighbourhood',
 '\nM1A\nNot assigned\nNot assigned',
 '\nM2A\nNot assigned\nNot assigned',
 '\nM3A\nNorth York\nParkwoods',
 '\nM4A\nNorth York\nVictoria Village',
 '\nM5A\nDowntown Toronto\nHarbourfront',
 '\nM5A\nDowntown Toronto\nRegent Park',
 '\nM6A\nNorth York\nLawrence Heights',
 '\nM6A\nNorth York\nLawrence Manor',
 "\nM7A\nQueen's Park\nNot assigned",
 '\nM8A\nNot assigned\nNot assigned',
 '\nM9A\nEtobicoke\nIslington Avenue',
 '\nM1B\nScarborough\nRouge',
 '\nM1B\nScarborough\nMalvern',
 '\nM2B\nNot assigned\nNot assigned',
 '\nM3B\nNorth York\nDon Mills North',
 '\nM4B\nEast York\nWoodbine Gardens',
 '\nM4B\nEast York\nParkview Hill',
 '\nM5B\nDowntown Toronto\nRyerson',
 '\nM5B\nDowntown Toronto\nGarden District',
 '\nM6B\nNorth York\nGlencairn',
 '\nM7B\nNot assigned\nNot assigned',
 '\nM8B\nNot assigned\nNot assigned',
 '\nM9B\nEtobicoke\nCloverdale',
 '\nM9B\nEtobicoke\nIslington',
 '\nM9B\nEtobicoke\nMartin Grove',
 '\nM9B\nEtobicoke\nPrincess Ga

Delete the rows containing _Not assigned_ value in the 2nd column and for those where _Not assigned_ is only in the 3rd column copy the value from the 2nd column:

In [9]:
data=[]
for value in values:
    value=value.split('\n')
    value.pop(0)
    if ('Not assigned' in value):
        if value.index('Not assigned')==2:
            value[2]=value[1]
            data.append(value)
        else:
            continue
    else:
        data.append(value)
        
headers=data.pop(0)
print(data)

[['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Harbourfront'], ['M5A', 'Downtown Toronto', 'Regent Park'], ['M6A', 'North York', 'Lawrence Heights'], ['M6A', 'North York', 'Lawrence Manor'], ['M7A', "Queen's Park", "Queen's Park"], ['M9A', 'Etobicoke', 'Islington Avenue'], ['M1B', 'Scarborough', 'Rouge'], ['M1B', 'Scarborough', 'Malvern'], ['M3B', 'North York', 'Don Mills North'], ['M4B', 'East York', 'Woodbine Gardens'], ['M4B', 'East York', 'Parkview Hill'], ['M5B', 'Downtown Toronto', 'Ryerson'], ['M5B', 'Downtown Toronto', 'Garden District'], ['M6B', 'North York', 'Glencairn'], ['M9B', 'Etobicoke', 'Cloverdale'], ['M9B', 'Etobicoke', 'Islington'], ['M9B', 'Etobicoke', 'Martin Grove'], ['M9B', 'Etobicoke', 'Princess Gardens'], ['M9B', 'Etobicoke', 'West Deane Park'], ['M1C', 'Scarborough', 'Highland Creek'], ['M1C', 'Scarborough', 'Rouge Hill'], ['M1C', 'Scarborough', 'Port Union'], ['M3C', 'North York', 'Flemingdon Park'

Merge the rows with the same value in the 1st column. (More than one neighborhood can exist in one postal code area. These rows are combined into one row with the neighborhoods separated with a comma.)


In [10]:
i=0

while i<len(data)-1:
    element=data[i]
    while element[0]==data[i+1][0]:
        element[2]=element[2]+', '+data[i+1][2]
        data.pop(i+1)
#        print(i, element, len(data))
        if (i==len(data)-1):
            break
#    print(i, '\n', element)
    i=i+1
print(data)

[['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Harbourfront, Regent Park'], ['M6A', 'North York', 'Lawrence Heights, Lawrence Manor'], ['M7A', "Queen's Park", "Queen's Park"], ['M9A', 'Etobicoke', 'Islington Avenue'], ['M1B', 'Scarborough', 'Rouge, Malvern'], ['M3B', 'North York', 'Don Mills North'], ['M4B', 'East York', 'Woodbine Gardens, Parkview Hill'], ['M5B', 'Downtown Toronto', 'Ryerson, Garden District'], ['M6B', 'North York', 'Glencairn'], ['M9B', 'Etobicoke', 'Cloverdale, Islington, Martin Grove, Princess Gardens, West Deane Park'], ['M1C', 'Scarborough', 'Highland Creek, Rouge Hill, Port Union'], ['M3C', 'North York', 'Flemingdon Park, Don Mills South'], ['M4C', 'East York', 'Woodbine Heights'], ['M5C', 'Downtown Toronto', 'St. James Town'], ['M6C', 'York', 'Humewood-Cedarvale'], ['M9C', 'Etobicoke', 'Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe'], ['M1E', 'Scarborough', 'Guildwood, Morningside, We

Create a dataframe with the columns: PostalCode, Borough, and Neighborhood

In [11]:
df = pd.DataFrame(data,columns=['PostalCode', 'Borough', 'Neighborhood'])
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


Use the .shape method to print the number of rows of the dataframe

In [12]:
df.shape

(103, 3)

In [22]:
dfcoordinates=pd.read_csv('https://cocl.us/Geospatial_data')
dfcoordinates.columns=['PostalCode','Latitude','Longitude']
dfcoordinates.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
result = pd.merge(df, dfcoordinates, on='PostalCode')
result.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [24]:
result.shape

(103, 5)